In [9]:
import pandas as pd
import numpy as np
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import CustomSelectionIntegrationStrategy
from troppo.methods.reconstruction.fastcore import FASTcore, FastcoreProperties

# parsing rule 
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

In [10]:
# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')
expression_data = pd.read_csv('/home/biodata/aman/expression_data_ensembl.csv', sep=',', index_col=0)

In [11]:
expression_data

,expression
gene,
ENSG00000223972,0.000000
ENSG00000227232,0.015341
ENSG00000278267,0.000000
ENSG00000243485,0.000000
ENSG00000284332,0.000000
...,...
ENSG00000198727,9560.073000
ENSG00000210195,14.681607
ENSG00000210196,2.561994


In [12]:
expression_data.T

gene,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,NaN,ENSG00000240361,ENSG00000186092,ENSG00000233750,...,ENSG00000210184,ENSG00000210191,ENSG00000198786,ENSG00000198695,ENSG00000210194,ENSG00000198727,ENSG00000210195,ENSG00000210196,ENSG00000275215,ENSG00000278233
expression,0.0,0.015341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046024,...,0.0,0.061365,856.93335,129.11223,1.488104,9560.073,14.681607,2.561994,0.0,0.0


In [13]:
#print(expression_data.head())
#expression_data_transposed = expression_data.T
#print(expression_data_transposed.head())

omics_container = TabularReader(path_or_df=expression_data.T, 
                                nomenclature='ensembl_gene_id',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]

In [14]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

In [15]:
data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)

In [16]:
from math import log
thres = 5 * log(2)

protected_reactions = ['MAR04413','MAR09931','MAR09932','MAR12140','MAR10024','MAR10062','MAR10063','MAR10064','MAR10065','MAR13082']

def integration_fx(reaction_map_scores):
    return [[k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > thres) or k in protected_reactions]]
    #return [k for k, v in reaction_map_scores.get_scores().items() if (v is not None and v > thres) or k in protected_reactions]

threshold_integration = CustomSelectionIntegrationStrategy(group_functions=[integration_fx])
threshold_scores = threshold_integration.integrate(data_map=data_map)

In [17]:
threshold_scores

[['MAR03905',
  'MAR03907',
  'MAR04097',
  'MAR04099',
  'MAR04108',
  'MAR04133',
  'MAR04281',
  'MAR04388',
  'MAR04283',
  'MAR08357',
  'MAR04379',
  'MAR04301',
  'MAR04355',
  'MAR04358',
  'MAR04363',
  'MAR04365',
  'MAR04368',
  'MAR04370',
  'MAR04371',
  'MAR04372',
  'MAR04373',
  'MAR04375',
  'MAR04381',
  'MAR04391',
  'MAR04394',
  'MAR04396',
  'MAR04521',
  'MAR06412',
  'MAR07745',
  'MAR07747',
  'MAR08360',
  'MAR08652',
  'MAR08757',
  'MAR03989',
  'MAR04122',
  'MAR04837',
  'MAR05395',
  'MAR05396',
  'MAR09727',
  'MAR05397',
  'MAR05398',
  'MAR05399',
  'MAR05400',
  'MAR05401',
  'MAR08568',
  'MAR08569',
  'MAR08570',
  'MAR08571',
  'MAR08572',
  'MAR08573',
  'MAR08574',
  'MAR08575',
  'MAR08576',
  'MAR08577',
  'MAR08578',
  'MAR08579',
  'MAR08581',
  'MAR08591',
  'MAR08592',
  'MAR08580',
  'MAR08582',
  'MAR08587',
  'MAR08589',
  'MAR08585',
  'MAR03944',
  'MAR04128',
  'MAR04130',
  'MAR04131',
  'MAR04132',
  'MAR04303',
  'MAR04414',
  'MAR

In [23]:
# Get the index of the reaction of the CORE reaction set
ordered_ids = {r:i for i,r in enumerate(model_wrapper.model_reader.r_ids)}
core_idx = [[ordered_ids[k] for k in l] for l in threshold_scores]

# Define the FastCORE properties
properties = FastcoreProperties(core=core_idx, solver='CPLEX')

# instantiate the FastCORE class
fastcore = FASTcore(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

# Run the algorithm
model_fastcore = fastcore.run()

J size9
[ 9117  9685  9686 11744 12721 12722 12723 12724 12725]
before LP7
LP7
-0.0005
done LP7
LP9
3145302.112223872
done LP9


Exception: Inconsistent irreversible core reactions 
	Impossible to build model

In [61]:
model_fastcore

NameError: name 'model_fastcore' is not defined

In [110]:
pd.DataFrame(model_fastcore)

""


In [111]:
# variable will contain the indices of the reactions that should be kept in the final model. 

In [112]:
print(type(model_fastcore))

<class 'list'>


In [113]:
pd.DataFrame(model_fastcore).to_csv('fastcore_output_tinit.tsv', sep='\t')

---

In [114]:
# model.reactions
# len(model.reactions)

In [115]:
selected_reactions = [model.reactions[i] for i in model_fastcore]
selected_reactions

[]

---

In [116]:
ctx_model = model.copy()

selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]

ctx_model.remove_reactions(to_remove, remove_orphans=True)

ctx_model

/home/biodata/miniconda3/envs/cplex_aman/lib/python3.10/site-packages/cobra/core/group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Name,HumanGEM
Memory address,7ac63efd7bb0
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,148
Objective expression,0
Compartments,


In [117]:
len(model.reactions), len(ctx_model.reactions)

(12971, 0)

In [118]:
# export
cobra.io.write_sbml_model(ctx_model, "fastcore_context_specific_model.xml")

In [119]:
solution = ctx_model.optimize()
print(solution.objective_value)

0.0


In [121]:
ctx_model.summary()

Non-linear or non-reaction model objective. Falling back to minimal display.


Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux
